<a href="https://colab.research.google.com/github/makhijakabir/machine-learning/blob/main/NNHandWrittenRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using Neural Networks to make a model that recognizes Hand Written Digits

##Imports

In [3]:
%load_ext tensorboard

In [4]:
from numpy.random import seed
seed(888)
from tensorflow.random import set_seed
set_seed(404)

In [5]:
import os
import numpy as np
import tensorflow as tf

##Constants

In [6]:
xTrainPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_xtrain.csv'
xTestPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_xtest.csv'
yTrainPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_ytrain.csv'
yTestPath = '/content/drive/MyDrive/Colab Notebooks/imageClassifyData/digit_ytest.csv'

##Get the Data

In [9]:
%%time

yTrainAll = np.loadtxt(yTrainPath, delimiter=',', dtype=int)
yTestAll = np.loadtxt(yTestPath, delimiter=',', dtype=int)

xTrainAll = np.loadtxt(xTrainPath, delimiter=',', dtype=int)
xTestAll = np.loadtxt(xTestPath, delimiter=',', dtype=int)

CPU times: user 52.1 s, sys: 2.64 s, total: 54.7 s
Wall time: 56 s
